In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd
import os
import re
import numpy as np
import time
import unidecode
from tqdm import tqdm
import csv

# Random integer for more realistic timing for clicks, buttons and searches during scraping
import random
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date
import json

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.31.1'

In [3]:
import undetected_chromedriver as uc


In [4]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import TimeoutException


In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Windows"', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'sec-fetch-site': 'same-site', 'sec-fetch-mode': 'navigate', 'sec-fetch-user': '?1', 'sec-fetch-dest': 'document', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US'}


In [7]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36


In [8]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [9]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = uc.ChromeOptions()

    # Going undercover:
    #option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    # option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
    
    return option

In [10]:
from undetected_chromedriver import Chrome

opts = create_option()
driver = Chrome(
    version_main=135,   # <-- khớp với “Current browser version is 135.xxxx”
    options=opts
)


driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
driver.get("https://www.tripadvisor.com")
driver.maximize_window()


In [11]:
# Tìm và nhấn vào tìm kiếm nhà hàng với XPath
restaurant_ele = driver.find_element(By.XPATH, "//a[@href='/Restaurants']")
restaurant_ele.click()

# Nhập thành phố
country = 'Singapore'
place = 'Singapore'
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# Tìm thanh search
search_bar_location = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[4]/div/div/div[2]/div/form/div[1]/div/input")

# Thêm thời gian trễ ngẫu nhiên giữa các hành động
time.sleep(random.uniform(0.8, 2))
search_bar_location.clear()

time.sleep(random.uniform(0.8, 2))
search_bar_location.send_keys(f"{place}")

# Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
actions = ActionChains(driver)
actions.move_to_element(search_bar_location).perform()
time.sleep(random.uniform(0.8, 2))

# Tìm liên kết "Hotels" và bấm vào
restaurant_link = driver.find_element(By.XPATH, "//div[@class='yOZMu u j']//a[1]")
time.sleep(random.uniform(2, 4))

# Bấm vào link "Hotels"
restaurant_link.click()
time.sleep(random.uniform(2, 4))




StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=135.0.7049.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0108D363+60275]
	GetHandleVerifier [0x0108D3A4+60340]
	(No symbol) [0x00EC06F3]
	(No symbol) [0x00ED20E1]
	(No symbol) [0x00ED11C0]
	(No symbol) [0x00EC78F3]
	(No symbol) [0x00EC5DFA]
	(No symbol) [0x00EC913A]
	(No symbol) [0x00EC91B7]
	(No symbol) [0x00F0A32A]
	(No symbol) [0x00EFE1EA]
	(No symbol) [0x00F2D08C]
	(No symbol) [0x00EFDCF4]
	(No symbol) [0x00F2D304]
	(No symbol) [0x00F4E6EB]
	(No symbol) [0x00F2CE86]
	(No symbol) [0x00EFC623]
	(No symbol) [0x00EFD474]
	GetHandleVerifier [0x012D8FE3+2467827]
	GetHandleVerifier [0x012D45E6+2448886]
	GetHandleVerifier [0x012EF80C+2560028]
	GetHandleVerifier [0x010A3DF5+153093]
	GetHandleVerifier [0x010AA3BD+179149]
	GetHandleVeri

In [12]:
# Hàm lấy thông tin URL trong trang
def getURL():
    page_source = BeautifulSoup(driver.page_source, 'html.parser') 
    restaurants = page_source.find_all('a', class_='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')
    all_restaurant_url = []

    
    for restaurant in restaurants:
        restaurant_link = restaurant.get('href')
        # Kiểm tra nếu không có dấu '#' trong URL
        if restaurant_link and '#' not in restaurant_link:
            restaurant_url = 'https://www.tripadvisor.com' + restaurant_link
            if restaurant_url not in all_restaurant_url:
                all_restaurant_url.append(restaurant_url)
                
    return all_restaurant_url

getURL()


['https://www.tripadvisor.com/Restaurant_Review-g294265-d10547345-Reviews-Curry_Magic-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d25152754-Reviews-Hopscotch_gardens_By_The_Bay-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d13505117-Reviews-Mr_Stork-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294264-d3856603-Reviews-Ocean_Restaurant-Sentosa_Island.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294264-d25271672-Reviews-CHIFA-Sentosa_Island.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d26206177-Reviews-The_Malayan_Council-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d17806127-Reviews-KOMA_Singapore-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d15123886-Reviews-SKAI-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d20150083-Reviews-Cappadocia_Turkish_Mediterranean_Restaurant-Singapore.html',
 'https://www.t

In [13]:
# Hàm lấy URL của tất cả các trang
URLs_all_page = []

def getUrlOnPages():
    input_page = 1000

    for page in range(input_page):
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Restaurant_Review' in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  
            driver.get(next_page_url)  
            time.sleep(random.uniform(1, 2))
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 379. Dừng scrape.


['https://www.tripadvisor.com/Restaurant_Review-g294265-d10547345-Reviews-Curry_Magic-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d25152754-Reviews-Hopscotch_gardens_By_The_Bay-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d13505117-Reviews-Mr_Stork-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294264-d3856603-Reviews-Ocean_Restaurant-Sentosa_Island.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294264-d25271672-Reviews-CHIFA-Sentosa_Island.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d26206177-Reviews-The_Malayan_Council-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d17806127-Reviews-KOMA_Singapore-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d15123886-Reviews-SKAI-Singapore.html',
 'https://www.tripadvisor.com/Restaurant_Review-g294265-d20150083-Reviews-Cappadocia_Turkish_Mediterranean_Restaurant-Singapore.html',
 'https://www.t

In [27]:
len(URLs_all_page)

8705

In [26]:
URLs_all_page = URLs_all_page[1517:]

In [28]:
# Kiểm tra xem file đã có dữ liệu chưa (để tránh ghi lại tiêu đề)
file_exists = os.path.isfile("Restaurant_Data\Restaurant_Data_SEA.csv")

with open(f"Restaurant_Data\Restaurant_Data_SEA.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    headers = [
        "Restaurant ID",
        "Country",
        "Tourist Attraction",
        "Restaurant Name",
        "Address",
        "Restaurant Website",
        "Hotline",
        "Ranking",
        "Price Type",
        "Opening and Closing hours",
        "Rating", 
        "Reviewers", 
        "Excellent Reviews",
        "Good Reviews",
        "Average Reviews",
        "Poor Reviews",
        "Terrible Reviews",
        "Description",
        "Cuisines",
        "Meal",
        "Special Diets",
        "Features",
        "Nearby Hotels",
        "Nearby Restaurants",
        "Nearby Attractions"
    ]
    
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)

    # Ghi tiêu đề chỉ khi file trống (chưa có dữ liệu)
    if not file_exists:
        writer.writeheader()

    # Khởi tạo biến đếm cho Restaurant ID
    restaurant_id_counter = 122542
    
    def serialize_nearby(lst):
        return "|".join(
            f"{i['name']}({i['rating']}/{i['review_count']}–{i['distance']})"
            for i in lst
        )
    
    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling restaurant data", dynamic_ncols=True, unit="restaurant", leave=False) as pbar:
        for restaurant_url in URLs_all_page:
            driver.get(restaurant_url)
            time.sleep(2)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        
            # Tên nhà hàng
            res_name = driver.find_element(By.CSS_SELECTOR, '.biGQs._P.hzzSG.rRtyp').text
            
            
            glance = soup.select_one("div.SZXBR.cZRjG")

            # 1) ADDRESS
            if glance:
                addr_el = soup.select_one("span[data-automation='restaurantsMapLinkOnName']")
                address = addr_el.get_text(strip=True) if addr_el else np.nan
            else:
                address = np.nan

            # 2) WEBSITE
            web_el = soup.select_one("a[data-automation='restaurantsWebsiteButton']")
            if web_el and web_el.has_attr("href"):
                website = web_el["href"]
            else:
                website = np.nan

            # 3) PHONE
            phone_el = soup.select_one("a[href^='tel:']")
            if phone_el:
                phone = phone_el.get_text(strip=True)
            else:
                phone = np.nan


            # mapping ngày → biến
            days_map = {
                "Sunday":  "open_sun",
                "Monday":  "open_mon",
                "Tuesday": "open_tue",
                "Wednesday":"open_wed",
                "Thursday": "open_thu",
                "Friday":   "open_fri",
                "Saturday": "open_sat",
            }

            # khởi tạo np.nan
            hours_data = {v: np.nan for v in days_map.values()}

            hours_section = soup.select_one('div[data-automation="hours-section"]')
            if hours_section:
                # duyệt từng row Day – Hours
                for row in hours_section.select('div.f.e.Q3 > div.f'):
                    # tên ngày
                    day_el = row.select_one('div.cGgaa .fOtGX')
                    if not day_el:
                        continue
                    day = day_el.get_text(strip=True)
                    # giờ: có thể nằm trong span.KxBGd hoặc span.fOtGX
                    time_el = row.select_one('span.KxBGd, span.fOtGX')
                    open_time = time_el.get_text(strip=True) if time_el else np.nan

                    var = days_map.get(day)
                    if var:
                        hours_data[var] = open_time

            # đẩy ra biến riêng
            open_sun = hours_data["open_sun"]
            open_mon = hours_data["open_mon"]
            open_tue = hours_data["open_tue"]
            open_wed = hours_data["open_wed"]
            open_thu = hours_data["open_thu"]
            open_fri = hours_data["open_fri"]
            open_sat = hours_data["open_sat"]

            # build a single string for opening hours
            hours_str = ";".join(
                f"{day}:{hours_data[var]}"
                for day, var in [
                    ("Sun","open_sun"), ("Mon","open_mon"),
                    ("Tue","open_tue"), ("Wed","open_wed"),
                    ("Thu","open_thu"), ("Fri","open_fri"),
                    ("Sat","open_sat")
                ]
            )


            # --- Ranking ---
            # 1) Chọn <span class="HUMGB bTeln"> chứa "#91 of 136 Restaurants..."
            diZaT = soup.select_one('span.diZaT')
            if diZaT:
                # separator=" " sẽ chèn dấu cách giữa các phần text
                ranking = diZaT.get_text(" ", strip=True)
            else:
                ranking = np.nan

            # --- Price level ---
            # 1) Chọn khối class chứa danh mục + price: <span class="HUMGB cPbcf">
            cat_block = soup.select_one('div.CsAqy span.HUMGB.cPbcf')
            price = np.nan
            if cat_block:
                # 2) Lấy tất cả <a> trong đó (lần lượt là cuisine1, cuisine2, ..., price)
                links = cat_block.find_all('a')
                # 3) Giá thường nằm ở <a> cuối cùng và có chứa ký hiệu $
                if links:
                    last = links[-1].get_text(strip=True)
                    if '$' in last:
                        price = last



            # --- Description ---
            desc_elem = soup.find(
                "div",
                class_=lambda c: c and "avBIb" in c.split() and "KxBGd" in c.split()
            )
            if desc_elem:
                description = " ".join(desc_elem.get_text(" ", strip=True).split())
            else:
                description = np.nan




            # --- Overall score & total reviews ---
            # Điểm trung bình (bubble score)
            # score_elem = soup.find("div", attrs={"data-automation": "reviewBubbleScore"})
            # overall_score = score_elem.get_text(strip=True) if score_elem else np.nan

            # # Tổng số review
            # count_elem = soup.find("div", attrs={"data-automation": "reviewCount"})
            # overall_review_count = count_elem.get_text(strip=True).strip("()") if count_elem else np.nan
            
            
            # excellent_reviews = good_reviews = average_reviews = poor_reviews = terrible_reviews = np.nan

            # # Duyệt các block của từng mức
            # for row in soup.select("div.YxspI div.jxnKb"):
            #     cat_elem = row.select_one("div.Ygqck.o.W.q")
            #     cat = cat_elem.get_text(strip=True) if cat_elem else None

            #     # Số lượng review tương ứng (ở div cuối cùng trong row)
            #     num_elem = row.select_one("div.q div.biKBZ")
            #     num = num_elem.get_text(strip=True) if num_elem else None

            #     if cat and num:
            #         num = num.replace(",", "")
            #         if cat == "Excellent":
            #             excellent_reviews = int(num)
            #         elif cat == "Good":
            #             good_reviews = int(num)
            #         elif cat == "Average":
            #             average_reviews = int(num)
            #         elif cat == "Poor":
            #             poor_reviews = int(num)
            #         elif cat == "Terrible":
            #             terrible_reviews = int(num)
            
            # --- Overall score & total reviews (HTML mới) ---
            
            
            # Điểm trung bình nằm trong <div class="biGQs _P fiohW uGzyt">
            score_elem = soup.select_one("div.qymjm div.uGzyt")
            overall_score = score_elem.get_text(strip=True) if score_elem else np.nan

            # Tổng số review nằm trong data-automation="bubbleReviewCount"
            revcount_elem = soup.select_one("div.qymjm [data-automation='bubbleReviewCount']")
            overall_review_count = revcount_elem.get_text(strip=True).strip("()") if revcount_elem else np.nan

            # --- Breakdown: Excellent, Good, Average, Poor, Terrible (HTML mới) ---
            excellent_reviews = good_reviews = average_reviews = poor_reviews = terrible_reviews = np.nan

            # Khung breakdown bây giờ là div.lMovV -> div.jxnKb
            breakdown = soup.select_one("div.lMovV")
            if breakdown:
                for row in breakdown.select("div.jxnKb"):
                    cat_elem = row.select_one("div.Ygqck.o.W.q")
                    num_elem = row.select_one("div.q div.biKBZ")
                    if not (cat_elem and num_elem):
                        continue
                    cat = cat_elem.get_text(strip=True)
                    num = int(num_elem.get_text(strip=True).replace(",", ""))
                    if cat == "Excellent":
                        excellent_reviews = num
                    elif cat == "Good":
                        good_reviews = num
                    elif cat == "Average":
                        average_reviews = num
                    elif cat == "Poor":
                        poor_reviews = num
                    elif cat == "Terrible":
                        terrible_reviews = num
                        

            nearby_results = {}

            for section in soup.find_all("div", {"data-automation": "bestNearByPOI"}):
                header = section.select_one("div.fiohW")
                if not header:
                    continue
                title = header.get_text(strip=True)                    
                category = title.replace("Best nearby ", "").capitalize()  

                items = []
                # 2) Duyệt từng box
                for box in section.select("div.BtnDR"):
                    # name
                    name_el = box.select_one("div.fiohW")
                    name = name_el.get_text(strip=True) if name_el else None

                    # rating
                    rat_el = box.select_one("[data-automation=bubbleRatingValue]")
                    rating = rat_el.get_text(strip=True) if rat_el else None

                    # review count
                    rev_el = box.select_one("[data-automation=bubbleReviewCount]")
                    if rev_el:
                        review_count = rev_el.get_text(strip=True).strip("()")
                    else:
                        review_count = None

                    # distance (chứa "away")
                    dist_el = next((
                        d for d in box.select("div.pZUbB") 
                        if d.get_text(strip=True).endswith("away")
                    ), None)
                    distance = dist_el.get_text(strip=True) if dist_el else None

                    items.append({
                        "name": name,
                        "rating": rating,
                        "review_count": review_count,
                        "distance": distance
                    })

                nearby_results[category] = items



            # --- 1) Mở rộng panel features (nếu có) ---
            cuisines = meal_types = special_diets = np.nan
            features = []
            try:
                see_all_btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//button[contains(., 'See all features')]")
                    )
                )
                see_all_btn.click()

                panel = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.WpMsY"))
                )

                # --- 2) Duyệt từng mục con trong panel ---
                rows = panel.find_elements(By.CSS_SELECTOR, "div.iPiKu")
                for row in rows:
                    title = row.find_element(By.CSS_SELECTOR, "div.UFJyF.Wf").text.strip().lower()

                    if "cuisines" in title:
                        cuisines = row.find_element(By.CSS_SELECTOR, "div.pZUbB.KxBGd").text.strip()
                    elif "meal type" in title:
                        meal_types = row.find_element(By.CSS_SELECTOR, "div.pZUbB.KxBGd").text.strip()
                    elif "special diet" in title:
                        special_diets = row.find_element(By.CSS_SELECTOR, "div.pZUbB.KxBGd").text.strip()
                    elif "features" in title:
                        feats = row.find_elements(By.CSS_SELECTOR, "div.pZUbB.avBIb.KxBGd")
                        features = [f.text.strip() for f in feats if f.text.strip()]
            except TimeoutException:
                # Không tìm thấy nút hoặc panel features → giữ nguyên giá trị mặc định
                pass

            # Expand features list into pipe separated
            features_str = "|".join(features)
                        
            # Tạo Restaurant ID
            restaurant_id = f'RS{str(restaurant_id_counter).zfill(6)}'
            restaurant_id_counter += 1


           
            writer.writerow({
                "Restaurant ID":            restaurant_id,
                "Country":                  country,
                "Tourist Attraction":       place,
                "Restaurant Name":          res_name,
                "Address":                  address,
                "Restaurant Website":       website,
                "Hotline":                  phone,
                "Ranking":                  ranking,
                "Price Type":               price,
                "Opening and Closing hours":hours_str,
                "Rating":                   overall_score,
                "Reviewers":                overall_review_count,
                "Excellent Reviews":        excellent_reviews,
                "Good Reviews":             good_reviews,
                "Average Reviews":          average_reviews,
                "Poor Reviews":             poor_reviews,
                "Terrible Reviews":         terrible_reviews,
                "Description":              description,
                "Cuisines":                 cuisines,
                "Meal":                     meal_types,
                "Special Diets":            special_diets,
                "Features":                 features_str,
                "Nearby Hotels":            serialize_nearby(nearby_results.get("Hotels", [])),
                "Nearby Restaurants":       serialize_nearby(nearby_results.get("Restaurants", [])),
                "Nearby Attractions":       serialize_nearby(nearby_results.get("Attractions", [])),
            })
            
            
            # Cập nhật thanh tiến độ sau mỗi nhà hàng
            pbar.update(1)
